<h1><center><b>Tropical Fishes</b></center></h1>
<h1><center><b>Scrapped Data with 10 classes-CE Loss Function</b></center></h1>
<center><b>15 March 2023</b></center><br/>
<center><b>ISNCC IEEE @HBKU</b></center><br/>
<center><b>Nauman Ullah Gilal</b></center><br/>
<center><b>Co-Authors: Fahad Majeed, Khaled Al-Thelaya, Mehak Khan, Jens Schneider, and Marco Agus</b></center><br/>
<center><b>IDEAL Lab</b></center><br/>
<center><b>College of Science and Engineering</b></center><br/>
<center><b>Hamad Bin Khalifa University<b></center><br/></h4>


In [ ]:
!nvidia-smi

# Data Preparation
Check training data

In [ ]:
!ls scrapped_dataset_new

In [ ]:
!ls test

# train_path = '/workspace/Nauman/MENA/MENA-v1-4C-Gym-Scheme/MENA-v1-c-109/MENA-v1-c-109/Training'
# test_path = '/workspace/Nauman/MENA/MENA-v1-4C-Gym-Scheme/MENA-v1-c-109/MENA-v1-c-109/Testing'

# Data set Categories, Classes, Labels

In [ ]:
import os
categories = os.listdir('scrapped_dataset_new')
categories

In [ ]:
TRAIN_DIR = "scrapped_dataset_new"

# Visualization of training samples

In [ ]:
# Visualize the training data
import matplotlib.pyplot as plt
import numpy as np
rows = 2
cols = 5
fig, ax = plt.subplots(rows, cols, figsize=(60,60))
fig.suptitle("Showing one random image from each training class", y=1.05, fontsize=40, weight = 'bold') # Adding  y=1.05, fontsize=24 helped me fix the suptitle overlapping with axes issue
data_dir = TRAIN_DIR
foods_sorted = sorted(os.listdir(data_dir))
food_id = 0
for i in range(rows):
  for j in range(cols):
    try:
      food_selected = foods_sorted[food_id] 
      food_id += 1
    except:
      break
    if food_selected == '.DS_Store':
        continue
    food_selected_images = os.listdir(os.path.join(data_dir,food_selected)) # returns the list of all files present in each food category
    food_selected_random = np.random.choice(food_selected_images) # picks one food item from the list as choice, takes a list and returns one random item
    img = plt.imread(os.path.join(data_dir,food_selected, food_selected_random))
    ax[i][j].imshow(img)
    ax[i][j].set_title(food_selected, pad = 10,weight = 'bold', color = 'red', fontsize = 30)
    
plt.setp(ax, xticks=[],yticks=[])
plt.tight_layout()
# https://matplotlib.org/users/tight_layout_guide.html


# Visualization of samples from Test set 1 

In [ ]:
# Visualize the training data
import matplotlib.pyplot as plt
import numpy as np
rows = 2
cols = 5
fig, ax = plt.subplots(rows, cols, figsize=(60,60))
fig.suptitle("Showing one random image from each training class", y=1.05, fontsize=40, weight = 'bold') # Adding  y=1.05, fontsize=24 helped me fix the suptitle overlapping with axes issue
data_dir = 'test'
foods_sorted = sorted(os.listdir(data_dir))
food_id = 0
for i in range(rows):
  for j in range(cols):
    try:
      food_selected = foods_sorted[food_id] 
      food_id += 1
    except:
      break
    if food_selected == '.DS_Store':
        continue
    food_selected_images = os.listdir(os.path.join(data_dir,food_selected)) # returns the list of all files present in each food category
    food_selected_random = np.random.choice(food_selected_images) # picks one food item from the list as choice, takes a list and returns one random item
    img = plt.imread(os.path.join(data_dir,food_selected, food_selected_random))
    ax[i][j].imshow(img)
    ax[i][j].set_title(food_selected, pad = 10,weight = 'bold', color = 'green', fontsize = 30)
    
plt.setp(ax, xticks=[],yticks=[])
plt.tight_layout()
# https://matplotlib.org/users/tight_layout_guide.html


# Declaration of Efficient Net family with Resolution, Epochs, Learning Rate, Batch size and Label smoothing values

In [ ]:
architecture = 'efficientnet-b0'
logfile = 'efficientnet-b0-MENA.csv'

## The tables
effnet_size = ({
    'efficientnet-b0':300,      # original resolution
    'efficientnet-b1':240,      # original resolution
    'efficientnet-b2':400,      # original resolution
    'efficientnet-b3':400,      # original resolution
    'efficientnet-b4':500,      # original resolution
    'efficientnet-b5':456,      # original resolution
    'efficientnet-b6':528,      # original resolution
    'efficientnet-b7':600,      # original resolution
    'efficientnet-lite0':224,
    'efficientnet-lite2':260,
    'efficientnet-lite4':384,
    'resnet18':224,
    'resnet34':224,
    'resnet50':224,
    'resnet101':224,
    'resnet152':224
})
batch_size = ({ 
    'efficientnet-b0':128,      # initially 160
    'efficientnet-b1':128,      #
    'efficientnet-b2':128,       #88 160
    'efficientnet-b3':16,       #
    'efficientnet-b4':16,       #
    'efficientnet-b5':14,       #
    'efficientnet-b6':16,
    'efficientnet-b7':8,
    'efficientnet-lite0': 160,
    'efficientnet-lite2': 160,
    'efficientnet-lite4':24,
    
    'resnet18':400,             # 3080: ok
    'resnet34':320,            # RTX 3090 rocz!
    'resnet50':256,             # 3080: ok
    'resnet101':96,
    'resnet152':64
})
learning_rate = ({
    'efficientnet-b0':5e-4,     # 2e-3: too large, 5e-4: too large
    'efficientnet-b1':2e-3,     # 1e-3: 82.53/93.9
    'efficientnet-b2':1e-3,     # 1e-3, 2e-3     
    'efficientnet-b3':2e-3,    
    'efficientnet-b4':2e-3,  
    'efficientnet-b5':1.5e-3,  
    'efficientnet-b6':4e-3,
    'efficientnet-b7':4e-3,
    'efficientnet-lite0': 5e-4,
    'efficientnet-lite2':1e-3,
    'efficientnet-lite4':5e-4,
    'resnet18':2.823e-02,
    'resnet34':-1,
    'resnet50':1.016e-02,
    'resnet101':-1,
    'resnet152':-1,
})
epochs = ({
    'efficientnet-b0':(1,100), # for initally top losses
    'efficientnet-b1':(6,20),
    'efficientnet-b2':(15,50),
    'efficientnet-b4':(5,25),
    'efficientnet-lite0': (15,80),
    'efficientnet-lite2': (5,25),
    'resnet18':(10,50),
    'resnet34':(10,50),
    'resnet50':(10,50),
    'resnet101':(10,50),
    'resnet152':(10,50)
})
label_smoothing = ({           # a = 0.00     a = 0.05     a = 0.10     a = 0.15     a = 0.20
    'efficientnet-b0':0.05,    #
    'efficientnet-b1':0.05,    #
    'efficientnet-b2':0.05,    # 0.05
    'efficientnet-b3':0.05,    #
    'efficientnet-b4':0.05,    #
    'efficientnet-b5':0.05,
    'efficientnet-b6':0.05,
    'efficientnet-b7':0.05,
    'efficientnet-lite0': 0.05,
    'efficientnet-lite2': 0.05,
    'resnet18':0.05,
    'resnet34':0.05,
    'resnet50':0.05,
    'resnet101':0.05,
    'resnet152':0.05
})
## Testing accuracies (top1,top5)
scores = ({
    'efficientnet-b0':(0,0),
    'efficientnet-b1':(0,0),
    'efficientnet-b2':(0,0),
    'efficientnet-b3':(0,0),
    'efficientnet-b4':(0,0),
    'efficientnet-lite0':(0,0),
    'efficientnet-lite2': (0,0),
    'resnet50':(0,0)
})
tta_scores = ({
    'efficientnet-b0':(0,0),
    'efficientnet-b1':(0,0),
    'efficientnet-b2':(0,0),
    'efficientnet-b3':(0,0),
    'efficientnet-b4':(0,0),
    'efficientnet-lite0':(0,0),
    'efficientnet-lite2': (0,0),
    'efficientnet-lite4':(0,0),
    'resnet50':(0,0)
})

 # Writing training data into csv

In [ ]:
from pathlib import Path
import os.path
train_dir = "scrapped_dataset_new"
training_path = Path(train_dir)

In [ ]:
import pandas as pd
filepaths = list(training_path.glob(r'**/*.jpg'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))
filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')
images = pd.concat([filepaths, labels], axis=1)
train_df = pd.DataFrame(images)
train_df

# Viualization of Training Data, Samples per class

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['axes.facecolor'] = 'white'
lbl = train_df['Label']
# print(lbl)
# print(lbl.size)
plt.figure(figsize=(70,20))
ax = sns.countplot(x= lbl, data=train_df)
ax.bar_label(ax.containers[0], weight = 'bold', fontsize = '30', color = 'red')
plt.xticks(rotation=90, fontsize = 40, weight = 'bold', color = 'black')
plt.yticks( fontsize = 40, weight = 'bold', color = 'black')
plt.xlabel("Labels", fontsize = 40, weight = 'bold', color = 'black')
plt.ylabel("Number of Images", fontsize = 40, weight = 'bold', color = 'black')

# Images Per Catergory

In [ ]:
counted = train_df.groupby(["Label"]).size()
print(counted)

# Test set with 50 Images per Class

In [ ]:
test_dir = "test"
test_path = Path(test_dir)
filepaths = list(test_path.glob(r'**/*.jpg'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))
filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')
images = pd.concat([filepaths, labels], axis=1)
test_df= pd.DataFrame(images)
test_df

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
lbl = test_df['Label']
# print(lbl)
# print(lbl.size)
plt.figure(figsize=(70,20))
ax = sns.countplot(x= lbl, data=test_df)
ax.bar_label(ax.containers[0], weight = 'bold', fontsize = '30', color = 'red')
plt.xticks(rotation=90, fontsize = 40, weight = 'bold', color = 'black')
plt.yticks( fontsize = 40, weight = 'bold', color = 'black')
plt.xlabel("Labels", fontsize = 40, weight = 'bold', color = 'black')
plt.ylabel("Number of Images", fontsize = 40, weight = 'bold', color = 'black')

# Import Fastai and pytorch Library

In [ ]:
from fastai.vision.all import *
from fastai import __version__
import torch
from torch.utils.data import Dataset, DataLoader
from efficientnet_pytorch import EfficientNet
from fastai.vision import *
from fastai import optimizer, losses, metrics
from functools import partial, wraps
import matplotlib.pyplot as plt
import random

# Fastai version checking

In [ ]:
import fastai
fastai.__version__
torch.__version__

# Torch version checking

In [ ]:
import torch
torch.cuda.current_device()
torch.cuda.device(0)
torch.cuda.device_count()
torch.cuda.get_device_name(0)
torch.cuda.is_available()


In [ ]:
# # batch size
# bs = 128
# # image resolution
# img_size = 224
model = EfficientNet.from_pretrained("efficientnet-b0", num_classes=10)

# Preparation of data for learner (Fastai learner)

In [ ]:
batch_tfms = [Zoom(),Rotate(), Flip(), Brightness(), Contrast(), Saturation()]

In [ ]:
bs   = batch_size[architecture]
imgs = effnet_size[architecture]
resize = (imgs*4)//3
data0 = (ImageDataLoaders.from_df(train_df, valid_pct=0.2,bs=bs,label_col=1,
                                        shuffle_train=True,
                                        item_tfms=Resize(imgs),
                                        batch_tfms = batch_tfms
                                 ))

print("Image size=", imgs)
print("Batch size=", bs)
print("Architecture=", architecture)
print(resize)


In [ ]:
data0.show_batch()

In [ ]:
loss_func = losses.CrossEntropyLossFlat()
model = EfficientNet.from_pretrained(architecture, num_classes=10)
learner_type = Learner

# Learner definition

In [ ]:
best_pth = 'tf_exp0_scrapped_dataset_b0'
checkpoints = SaveModelCallback(fname=best_pth,monitor='accuracy',comp=np.greater, with_opt=True)
# brainfreeze = BnFreeze()
learn = ( learner_type(data0, model,metrics=[accuracy],loss_func=loss_func,
                        cbs=[ShowGraphCallback(),checkpoints]).to_fp16())
print("Best pth is=", best_pth)

# How do we choose learning rates?
If the learning rate is too slow, training will take a lot of time. If the learning rate is too high, 
we will be jumping around the minimum loss, getting farther and farther from the minimum and never reach it.
So to start the experiment of finding a learning rate, we train the model while increasing the learning rate.
Then we plot the loss against the learning rate, and stop when the loss starts to explode.

In [ ]:
learn.lr_find()
learn.recorder.plot_lr_find()

In [ ]:
low_lr = 1e-4
lr = 1e-3
learn.fit_one_cycle(1 , lr_max = slice(low_lr, lr))

# Unfreeze the Learner
fast.ai freezes the pre-trained part when we create the model. So up to this point, 
we only trained the last layer block. Next, we will unfreeze the pre-trained part and train the whole model.

In [ ]:
learn.unfreeze()

In [ ]:
low_lr = 1e-4
lr = 1e-3
learn.fit_one_cycle(1, lr_max = slice(low_lr, lr))

# Test(%) on Test set 1 (50 Images per class)

In [ ]:
data_test = (ImageDataLoaders.from_df(test_df, valid_pct = 0.0, splitter=None, shuffle=False, label_col=1, item_tfms=Resize(imgs)))
preds = learn.get_preds(dl=data_test)
preds
print("length of preds[1]",len(preds[1]))
acc= accuracy(preds[0], preds[1])
print(" BaselineTop-1 Accuracy:", acc)

In [ ]:
err = {}
log_preds, y  = learn.tta(dl=data_test)
tta_acc = accuracy(log_preds, y)
print(tta_acc)
err[0] = (100.0, 100.0*(1.0-float(tta_acc)))

In [ ]:
inter_test = ClassificationInterpretation.from_learner(learn, dl =data_test)
inter_test.plot_confusion_matrix(figsize = (10,10))

In [ ]:
PATH="models/tf_exp0_scrapped_dataset.p"
torch.save(model.state_dict(), PATH)

# Saving best pth with baseline for future using and for scheme.

In [ ]:
best_pth

In [ ]:
import shutil
source = 'models/'+best_pth+'.pth'
destination = 'models/baseline/'+best_pth
# Copy the file from source to destination
shutil.copy(source, destination)

In [ ]:
from fastai.vision.all import *
from fastai.vision.widgets import ImageClassifierCleaner
import random as rn
img_size = imgs
drop_idxx = []
los = []
top_losses = []
torch.manual_seed(42)
def class_frequencies(data,class_key='Label'):
    classes = set(data[class_key])
    class_freq = {}
    max_freq = 0.5
    min_freq =10000000#1000000
    for cl in classes:
        class_df = data[data[class_key]==cl]
        freq = len(class_df)
        max_freq = max(freq,max_freq)
        min_freq = min(freq,min_freq)
        class_freq[cl] = freq
        #print(f'Frequency of class {cl}: {class_freq[cl]} ')
    print(f'Max frequency is {max_freq}')
    print(f'Min frequency is {min_freq}')
    return class_freq,max_freq,min_freq
    

def russian_roulette(idx, losses, data,class_key='Label', min_prob=0.5,max_prob=0.9): #min_prob=0.5,max_prob=0.9 original values
    
    class_freq,max_freq,min_freq = class_frequencies(data,class_key)
    min_prob = float(min_freq/max_freq)*max_prob
    print(f'Min probability = {min_prob}')
    deltap = (max_prob - min_prob)/(max_freq**2)
#     deltap = 0.0
#     if max_prob > min_prob:
#         deltap = (max_prob - min_prob)/(max_freq-min_freq)
            
    
    drop_idx = []
    for n,i in enumerate(idx.numpy()):
        l = data.iloc[i][class_key]
#         print("+++++++++ L", l)
#         prob = min_prob + deltap * (class_freq[l]**2)
        prob = min_prob + (class_freq[l]-min_freq)*deltap
        r = rn.random()
#         print("rrr",r, "prob+++", prob)
        if  r < prob:
            drop_idx.append(i)
            class_freq[l] -= 1 
#     print(class_freq)# returns number of images per class
    return drop_idx

def data_definition(data,img_size,tfms,model,best_pth,k_samples=100, min_prob=0.4, max_prob = 0.9, largest=True): # k_sample = 1000
   
    databunch = ( ImageDataLoaders.from_df(data, valid_pct=0.0, bs=bs,label_col=1,
                                        shuffle_train=True,
                                        item_tfms=Resize(imgs),
                                        batch_tfms = batch_tfms))

#     
    learn_cln = (learner_type(databunch, model,metrics=[accuracy],loss_func=loss_func, cbs=[ShowGraphCallback(),checkpoints]).to_fp16())
    learn_cln.load(best_pth)
    entire_training_set_csv0 = (ImageDataLoaders.from_df(data, valid_pct = 0.0, splitter=None, shuffle=False, label_col=1, item_tfms=Resize(imgs)))
#     entire_training_set_learner0 = (learner_type(entire_training_set_csv0, model, metrics=[accuracy]).to_fp16())
#     entire_training_set_learner0.load(best_pth)
    interp = ClassificationInterpretation.from_learner(learn_cln, dl = entire_training_set_csv0)
    interp.plot_top_losses(100, figsize = (11,15))
    losses = None
    idx = None
    if hybrid:
        l_big,idx_big = interp.top_losses(k_samples//2,largest=True)
        l_sm,idx_sm = interp.top_losses(k_samples//2,largest=False)
        losses = torch.cat((l_big,l_sm),0)
        idx = torch.cat((idx_big,idx_sm),0)
    else:
        losses,idx = interp.top_losses(k_samples,largest=largest)
    drop_idx = russian_roulette(idx,losses,data,min_prob = min_prob, max_prob = max_prob)
    top_losses = pd.DataFrame(data.iloc[drop_idx])
    top_losses = top_losses.to_csv('top_losses'+str(n+1)+'.csv', index = False)
    data_filtered = data.drop(data.index[drop_idx])
    print(f'Dropped {len(drop_idx)} top losses')
    return data_filtered


In [ ]:

### Gym training scheme --> 
### Threek kind of programs: mass, power or definition

drop_idxx = []
los = []
top_losses = []
n_training_series = 10
min_epochs = 1
min_prob=0.5
max_prob=0.9
delta_epochs_per_serie =  0
min_lr = 1e-4
low_lr = 1e-4
lr = 1e-3
kappa_lr = 1 
k_samples = 500
data_per_serie = {}
data_per_serie[0] = train_df
largest = True
from_scratch = False
hybrid = False
cleaning_cycles = 3


# First program: slimming --> Reducing data weight, increasing number of epochs, and decreaseing learning rate
for c in range(cleaning_cycles):  
    for n in range(n_training_series):
        print(f'+++++ Training serie: inverse pyramid {n} ++++++++')
        epochs_per_serie = min_epochs + n * delta_epochs_per_serie
#         lr = min_lr * kappa_lr * (n_training_series - n)
#         print("dynamic LR", dynamic_lr)
        data_per_serie[n+1] = data_definition(data_per_serie[n],img_size,batch_tfms,model,best_pth,k_samples=k_samples, min_prob = min_prob, max_prob=max_prob)
        print("************Round: ",n)
        print("************data per serie:******************")
    #     data_per_serie[n+1]
        print("************length of data per serie:******************")
        print(len(data_per_serie[n+1]))
        
        databunch = ( ImageDataLoaders.from_df(data_per_serie[n+1], valid_pct = 0.2,bs=bs,label_col=1,
                                        shuffle_train=True,
                                        item_tfms=Resize(imgs),
                                        batch_tfms = batch_tfms))

    #     learn = ( learner_type(databunch, model,metrics=[accuracy,top_5],loss_func=loss,cbs=[ShowGraphCallback(),history]).to_fp16())
        learn =(learner_type(databunch, model,metrics=[accuracy],loss_func=loss_func,
                            cbs=[ShowGraphCallback(),checkpoints]).to_fp16())
#         learn.split( lambda m: (model._conv_head) )
        learn.unfreeze() 
        learn.load(best_pth)
        learn.fit_one_cycle(epochs_per_serie, lr_max = slice(low_lr, lr))
        if n == 4 or n == 9:
            log_preds, y = learn.tta(dl=data_test)
            print("************Round: ",n)
            tta_acc = accuracy(log_preds, y)
            print('TTA Accuracy', tta_acc)
            err[c*n_training_series + n] = ( 100.0 *float(1.0 - len(data_per_serie[n+1])/len(data_per_serie[0])) ,  100.0 * (1.0 - float(tta_acc)))
            print(f'cycle {c} Error Rate is [{err}]')
            inter_test = ClassificationInterpretation.from_learner(learn, dl =data_test)
            inter_test.plot_confusion_matrix(figsize = (10,10))
            PATH="models/tropical_fishes_b0_"+str(c)+"_"+str(n)+".p"
            torch.save(model.state_dict(), PATH)
            csv = data_per_serie[n+1].to_csv("tropical_fishes_clean_csv"+str(c)+"_"+str(n)+".csv", index = False)
        

In [ ]:
err #

# Data Visualization after cleaning

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['axes.facecolor'] = 'white'
for i in range(21):
    train_df = data_per_serie[i]
    make_csvs = train_df.to_csv('cleaned_csv_'+str(i)+'.csv', index = False)
    lbl = train_df['Label']
    plt.figure(figsize=(70,20))
    ax = sns.countplot(x= lbl, data=train_df)
    ax.bar_label(ax.containers[0], weight = 'bold', fontsize = '30', color = 'red')
    plt.xticks(rotation=90, fontsize = 40, weight = 'bold', color = 'black')
    plt.yticks( fontsize = 40, weight = 'bold', color = 'black')
    plt.xlabel("Labels", fontsize = 40, weight = 'bold', color = 'black')
    plt.ylabel("Number of Images", fontsize = 40, weight = 'bold', color = 'black')